# Jupyter Notebook-based user interface

## Importing the needed libraries

In [ ]:
import rospy
import actionlib
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningFeedback, PlanningResult
from assignment_2_2023.msg import CustomMessage
from nav_msgs.msg import Odometry
import jupyros as jr
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D



## Node initialization and Declaration of pusblisher.

In [ ]:
rospy.init_node('action_client_node', anonymous=True)

In [ ]:
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)

client.wait_for_server()

custom_pub = rospy.Publisher('/custom_topic', CustomMessage, queue_size=10)

## Widgets for send/cancel Goal and display the output

In [ ]:
x_widget = widgets.FloatText(description='Target X:')
y_widget = widgets.FloatText(description='Target Y:')
send_button = widgets.Button(description='Set Goal')
cancel_button = widgets.Button(description='Cancel Goal')
feedback_output = widgets.Output()

## Callback function for sending goal Buttons

In [ ]:
def set_goal(x, y):
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    client.send_goal(goal, done_cb=goal_done_callback)
    with feedback_output:
        print("Goal sent with coordinates (x={}, y={}).\n".format(x, y))

# Callback function when a goal is completed
def goal_done_callback(status, result):
    with feedback_output:
        if status == actionlib.GoalStatus.SUCCEEDED:
            print("Goal succeeded.\n")
            reached_x.append(x_widget.value)
            reached_y.append(y_widget.value)
        elif status == actionlib.GoalStatus.PREEMPTED:
            print("Goal preempted.\n")
            canceled_x.append(x_widget.value)
            canceled_y.append(y_widget.value)
        else:
            print("Goal failed.\n")
            canceled_x.append(x_widget.value)
            canceled_y.append(y_widget.value)

def on_send_button_clicked(b):
    set_goal(x_widget.value, y_widget.value)

def on_cancel_button_clicked(b):
    client.cancel_goal()
    # Update canceled target lists
    canceled_x.append(x_widget.value)
    canceled_y.append(y_widget.value)
    with feedback_output:
            print("Goal canceled.\n")

send_button.on_click(on_send_button_clicked)
cancel_button.on_click(on_cancel_button_clicked)

## Displaying the robot's position and the reached/non- reached targets

In [ ]:
# Matplotlib figure setup
%matplotlib notebook
fig, ax = plt.subplots()
ln, = plt.plot([], [], 'ro')
x_data, y_data = [], []  # Robot trajectory
reached_x, reached_y = [], []  # Reached targets
canceled_x, canceled_y = [], []  # Canceled targets


# Legend elements
handles = [
    Line2D([], [], color='blue', label='Robot Trajectory', linestyle='-'),
    Line2D([], [], marker='s', markersize=6, color='green', markerfacecolor='green', linestyle='None', label='Reached'),
    Line2D([], [], marker='x', markersize=6, color='red', markerfacecolor='red', linestyle='None', label='Canceled'),
]

labels = [handle.get_label() for handle in handles]

# Function to initialize the plot
def plot_init():
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    legend = plt.legend(handles, labels, loc='upper left')
    legend.set_title('Legend')
    return ln, legend

# Function to update the plot
def update_plot(frame):
    ln.set_data(x_data, y_data)
    ln.set_color((0, 0, 0.8))  # Light shade of blue
    ln.set_linewidth(0.5)
    
    scatter_reached = plt.scatter(reached_x, reached_y, marker='s', color='green', zorder=2)
    scatter_canceled = plt.scatter(canceled_x, canceled_y, marker='x', color='red', zorder=1)
    
    ln.set_zorder(0)  # Set z-order of the trajectory line lower than the markers
    return ln, scatter_reached, scatter_canceled




# Callback for odometry messages
def odom_callback(odom_msg):
    x = odom_msg.pose.pose.position.x
    y = odom_msg.pose.pose.position.y
    vel_x = odom_msg.twist.twist.linear.x
    vel_z = odom_msg.twist.twist.angular.z

    custom_msg = CustomMessage()
    custom_msg.x = x
    custom_msg.y = y
    custom_msg.vel_x = vel_x
    custom_msg.vel_z = vel_z
    custom_pub.publish(custom_msg)
    
    x_data.append(x)
    y_data.append(y)


In [ ]:
rospy.Subscriber('/odom', Odometry, odom_callback)
display(x_widget, y_widget, send_button, cancel_button, feedback_output)

# Animation for the plot
ani = FuncAnimation(fig, update_plot, init_func=plot_init)
plt.show(block=True)